In [25]:
import boto3, json, os
import pandas as pd
from botocore.config import Config

In [12]:
S3_BUCKET = "hab-ree-data-json"
S3_PREFIX = "demanda-real"
OUTPUT_KEY = "merged/ree-data_full.json"

In [13]:
with open("credentials/aws_keys.json") as f:
    keys = json.load(f)

os.environ["AWS_ACCESS_KEY_ID"] = keys["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = keys["AWS_SECRET_ACCESS_KEY"]
os.environ["AWS_DEFAULT_REGION"] = keys["AWS_DEFAULT_REGION"]

In [14]:
s3 = boto3.client("s3")

In [37]:
# Bloque test
dfs = []
processed = 0
MAX_FILES = 5  # número de archivos de prueba

paginator = s3.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=S3_BUCKET, Prefix=S3_PREFIX):
    for obj in page.get("Contents", []):
        key = obj["Key"]
        print(obj)

        #body = s3.get_object(Bucket=S3_BUCKET, Key=key)["Body"].read()
        #data = json.loads(body)

        # Soporta dict con indicator.values o lista directa
        #values = data["indicator"]["values"] if isinstance(data, dict) else data
        #df = pd.DataFrame(values)
        #dfs.append(df)

        processed += 1
        print(f"[{processed}] {key} -> {len(df)} filas")

        if processed >= MAX_FILES:
            break
    if processed >= MAX_FILES:
        break

{'Key': 'demanda-real/2014/01-enero/ree-data_20140101.json', 'LastModified': datetime.datetime(2025, 9, 10, 8, 48, 33, tzinfo=tzutc()), 'ETag': '"3e3e46a8ee51c6abd366dfb57c7da67a"', 'ChecksumAlgorithm': ['CRC32'], 'ChecksumType': 'FULL_OBJECT', 'Size': 30026, 'StorageClass': 'STANDARD'}
[1] demanda-real/2014/01-enero/ree-data_20140101.json -> 139 filas
{'Key': 'demanda-real/2014/01-enero/ree-data_20140102.json', 'LastModified': datetime.datetime(2025, 9, 10, 8, 48, 34, tzinfo=tzutc()), 'ETag': '"d9f4b81bdf7374fa2e976db1965d480f"', 'ChecksumAlgorithm': ['CRC32'], 'ChecksumType': 'FULL_OBJECT', 'Size': 30026, 'StorageClass': 'STANDARD'}
[2] demanda-real/2014/01-enero/ree-data_20140102.json -> 139 filas
{'Key': 'demanda-real/2014/01-enero/ree-data_20140103.json', 'LastModified': datetime.datetime(2025, 9, 10, 8, 48, 34, tzinfo=tzutc()), 'ETag': '"f7418e4da8ab0b2297f03ca50c646c73"', 'ChecksumAlgorithm': ['CRC32'], 'ChecksumType': 'FULL_OBJECT', 'Size': 30026, 'StorageClass': 'STANDARD'}
[3

In [ ]:
# Bloque test
if dfs:
    sample_df = pd.concat(dfs, ignore_index=True)
    display(sample_df.head(10))

In [30]:
dfs = []
paginator = s3.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=S3_BUCKET, Prefix=S3_PREFIX):
    for obj in page.get("Contents", []):
        key = obj["Key"] 

        body = s3.get_object(Bucket=S3_BUCKET, Key=key)["Body"].read()
        data = json.loads(body)

        values = data["indicator"]["values"] if isinstance(data, dict) else data
        dfs.append(pd.DataFrame(values))

In [31]:
full_df = pd.concat(dfs, ignore_index=True)
print("Filas totales:", len(full_df))

Filas totales: 760090


In [33]:
# Probar también con orient="split" https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_json.html

payload = full_df.to_json(orient="records")
s3.put_object(
    Bucket=S3_BUCKET,
    Key=OUTPUT_KEY,
    Body=payload,
    ContentType="application/json"
)

print(f"Merged listo: s3://{S3_BUCKET}/{OUTPUT_KEY}")

Merged listo: s3://hab-ree-data-json/merged/ree-data_full.json
